# Imports

In [3]:
%pip install tweepy --upgrade
%pip install emoji

import tweepy
import random
import configparser
import numpy as np
from collections import Counter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Download and Clean Tweets

In [31]:
config = configparser.ConfigParser()
config.read('config.ini')

tokens = config['Tokens']

client = tweepy.Client(
  bearer_token=tokens['BearerToken'],
  consumer_key=tokens['ConsumerKey'],
  consumer_secret=tokens['ConsumerSecret'],
  access_token=tokens['AccessToken'],
  access_token_secret=tokens['AccessTokenSecret'],
)

In [ ]:
tweets = []
users = [
  'afraidofwasps',
  'ameliaelizalde',
  'boss_on_here',
  'darth_erogenous',
  'dril',
  'i_zzzzzz',
  'laserboat999',
  'len0killer',
  'Liv_Agar',
  'lunch_enjoyer',
  'nibiru_TRUTH',
  'OkButStill',
  'oldfriend99',
  'peterxinping',
  'pizza_jones',
  'RadishHarmers',
  'rajat_suresh',
  's4m31p4n',
  'Senn_Spud',
  'yesitsmyaccount',
  'ZeroSuitCamus'
]

# get twitter ids from usernames
user_data = client.get_users(usernames=users)
user_ids = list(map(lambda x: x['id'], user_data[0]))

# get tweets for each user id
for user_id in user_ids:
  users_tweets = []
  until_id = None
  for _ in range(50):
    users_tweets = client.get_users_tweets(
      user_id,
      exclude=['retweets', 'replies'], 
      max_results=100,
      until_id=until_id,
    )
    if users_tweets[0]:
      until_id = users_tweets[0][-1]['id']
      tweets += list(map(lambda x: x['text'], users_tweets[0]))

In [12]:
def print_tweets(tweets, indices):
    for tweet in np.array(tweets)[indices]:
        if len(tweet) > 0:
            print(tweet)
            print()

random_indices = list(np.random.choice(len(tweets), size=10, replace=False))

print_tweets(tweets, random_indices)
print(f'TOTAL TWEETS: {len(tweets)}')

(texting gf) In uber. Be home soon. Cant wait to see you (accidentally pressing dictation button) Ohhhh i want a hamburger so bad. Hot dog too. Ohh man I want a mcchicken. me too. Woww I want a burger. Yeah I want a cheeseburger too. Ohhh wow me too. I want a hot dog.With the bun

in my opinion we should legalize freedom

when the hand holding is so good you arrive ya man's native city 🔥🤤 https://t.co/P8956BHogA

My parents dog is whining. what do you have to whine about. Have you ever seen a man die

Whats fucked up is , some will think this is good!!! https://t.co/2samsGi56q

cuomo: just wanted to remind everyone i am a muslim. i am a jew. I am black. i am gay. goodbye https://t.co/K2hnurWEwA

if you remove the license plate frame that advertises your dealership youre a nasty son of a bitch &amp; you want small business owners to choke

thinking about doing a tweet that goes viral on smart twitter. might spend the next couple days just contemplating shit... HARD. watch this space 🧠



In [59]:
np.save('weird_tweets.npy', np.array(tweets))

In [4]:
tweets = list(np.load('weird_tweets.npy'))

In [13]:
def clean_tweet(tweet):
  tweet = remove_links(tweet)
  tweet = separate_symbols(tweet)
  return tweet

def remove_links(tweet):
  split_tweet = tweet.split()
  split_tweet = [split_str for split_str in split_tweet if split_str if 'https://' not in split_str]
  tweet = ' '.join(split_tweet)
  return tweet

def separate_symbols(tweet):
  tweet_chars = [f' {char} ' if not char.isalnum() and char != ' ' else char for char in tweet]
  tweet = ''.join(tweet_chars)
  return tweet

cleaned_tweets = list(map(clean_tweet, tweets))

print_tweets(cleaned_tweets, random_indices)

 ( texting gf )  In uber .  Be home soon .  Cant wait to see you  ( accidentally pressing dictation button )  Ohhhh i want a hamburger so bad .  Hot dog too .  Ohh man I want a mcchicken .  me too .  Woww I want a burger .  Yeah I want a cheeseburger too .  Ohhh wow me too .  I want a hot dog . With the bun

in my opinion we should legalize freedom

when the hand holding is so good you arrive ya man ' s native city  🔥  🤤 

My parents dog is whining .  what do you have to whine about .  Have you ever seen a man die

Whats fucked up is  ,  some will think this is good !  !  ! 

cuomo :  just wanted to remind everyone i am a muslim .  i am a jew .  I am black .  i am gay .  goodbye

if you remove the license plate frame that advertises your dealership youre a nasty son of a bitch  & amp ;  you want small business owners to choke

thinking about doing a tweet that goes viral on smart twitter .  might spend the next couple days just contemplating shit .  .  .  HARD .  watch this space  🧠 



# Create N-gram Model

$P(word_i \mid word_{i-n+1},...,word_{i-1})=\dfrac{Count(word_{i-n+1},...,word_{i-1},word_i)}{Count(word_{i-n+1},...,word_{i-1})}$

In [14]:
START = '<s>'
STOP = '</s>'

class TweetModel:
  def __init__(self, tweets, n):
    self.n = n
    self.counts = Counter()
    self.process_tweets(tweets)

  def process_tweets(self, tweets):
    tokenized_tweets = [tweet.split() for tweet in tweets]
    for tweet in tokenized_tweets:
      if len(tweet) > 0:
        tweet_ngrams = self.create_ngrams(tweet)
        for ngram in tweet_ngrams:
          self.counts[ngram] += 1

  def random_tweet(self):
    start_context = [START] * (self.n - 1)
    curr_context = start_context
    tweet = []
    while True:
      token = self.random_token(tuple(curr_context))
      if token == STOP:
        break
      tweet.append(token)
      curr_context.pop(0)
      curr_context.append(token)
    return ' '.join(tweet)

  def random_token(self, context):
    tokens = [token for (context1, token), _ in self.counts.items() if context == context1]
    rand = random.random()
    random_token = None
    total = 0
    for token in tokens:
      random_token = token
      total += self.get_probability(context, token)
      if total > rand:
        break
    return random_token

  def create_ngrams(self, tokens):
    ngrams = []
    for i in range(len(tokens) + 1):
        context = []
        for j in range(self.n - 1, 0, -1):
            if i - j < 0:
                context.append(START)
            else:
                context.append(tokens[i - j])
        if i == len(tokens):
            ngrams.append((tuple(context), STOP))
        else:
            ngrams.append((tuple(context), tokens[i]))
    return tuple(ngrams)

  def get_probability(self, context, token):
    denominator = 0
    for (context1, _), count in self.counts.items():
      if context == context1:
        denominator += count
    return self.counts[(context, token)] / denominator

  def print_counts(self, n):
    counts = self.counts.items()
    counts = sorted(counts, key=lambda x: x[1], reverse=True)
    for count in counts[:n]:
      print(count)

In [15]:
model = TweetModel(tweets=cleaned_tweets, n=3)
model.print_counts(20)

((('<s>', '<s>'), 'I'), 947)
((('.', '.'), '.'), 537)
((('!', '!'), '!'), 399)
((('<s>', '<s>'), 'i'), 355)
((('I', '’'), 'm'), 331)
((('<s>', '<s>'), 'The'), 252)
((('I', "'"), 'm'), 249)
((('it', '’'), 's'), 213)
((('<s>', '<s>'), '('), 208)
((('don', '’'), 't'), 199)
((('.', '.'), '</s>'), 173)
((('<s>', '<s>'), 'the'), 172)
((('<s>', '<s>'), 'if'), 166)
((('😉', '😉'), '😉'), 166)
((('don', "'"), 't'), 159)
((('<s>', '<s>'), 'My'), 158)
((('!', '!'), '</s>'), 156)
((('it', "'"), 's'), 142)
((('<s>', '<s>'), 'You'), 138)
((('<s>', '<s>'), 'It'), 136)


# Generate tweets

In [16]:
def print_tweet(tweet):
  print('+------------------------------------------------------------+')
  print('| +------+                                                   |')
  print('| |  /\  | N-Gram Bot @ngrambot - 12h                        |')
  print('| | [OO] |                                                   |')
  print('| |  []  |                                                   |')
  print('| +------+                                                   |')
  print('|                                                            |')

  for line in tweet_to_lines(tweet):
    padding = ' ' * (59 - len(line))
    print('| ' + line + padding + '|')

  print('|                                                            |')
  print('| <3 31.4k                                                   |')
  print('+------------------------------------------------------------+')

def tweet_to_lines(tweet):
  lines = []
  curr_line = ''
  for token in tweet.split():
    if len(curr_line) + len(token) + 1 <= 59:
      curr_line += token + ' '
    else:
      lines.append(curr_line)
      curr_line = token + ' '
  lines.append(curr_line)
  return lines

In [17]:
for i in range(10):
  print_tweet(model.random_tweet())

+------------------------------------------------------------+
| +------+                                                   |
| |  /\  | N-Gram Bot @ngrambot - 12h                        |
| | [OO] |                                                   |
| |  []  |                                                   |
| +------+                                                   |
|                                                            |
| My parents are crying with joy and relief . We have Water  |
| Monsters . Stop crying . You wake up I am shot in the      |
| garage . calm down lol yes , and then go home              |
|                                                            |
| <3 31.4k                                                   |
+------------------------------------------------------------+
+------------------------------------------------------------+
| +------+                                                   |
| |  /\  | N-Gram Bot @ngrambot - 12h                  